In [1]:
import pandas as pd
import os

# Vérification des données

## Fonctions de travail

In [ ]:
var_interet = [
    "tourney_date", "tourney_id", "tourney_name", "tourney_level", "surface",
    "score", "round", "match_num", "minutes", "winner_id", "winner_rank",
    "winner_ioc", "winner_name", "winner_hand", "loser_id", "loser_rank",
    "loser_ioc", "loser_name", "loser_hand", "w_bpSaved", "w_bpFaced",
    "l_bpSaved","l_bpFaced"
    ]

In [ ]:
def recup_match(type, genre, ind_deb, ind_fin):
    """Récupère les matches de tennis simples pour un genre
    et une période spécifiés."""

    data_dict = {}
    formats_colonnes = {}

    if genre == 'atp':
        liste_fichier = os.listdir('../donnees_tennis/ATP')
    else:
        liste_fichier = os.listdir('../donnees_tennis/WTA')

    for fichier in liste_fichier:
        nom_deb = type + ind_deb
        nom_fin = type + ind_fin

        if fichier.startswith(nom_deb) or fichier.startswith(nom_fin):
            file_path = (
                f"../donnees_tennis/{'ATP' if genre == 'atp' else 'WTA'}/{fichier}"
            )
            data = pd.read_csv(file_path, index_col=False)

            # Stockage du DataFrame
            data_dict[fichier] = data

            # Récupérer le format des colonnes pour ce DataFrame
            format_actuel = data.dtypes.to_dict()
            formats_colonnes[fichier] = format_actuel

    return data_dict, formats_colonnes

In [ ]:
def comp_format(formats_colonnes):
    """Compare les formats de colonnes entre plusieurs DataFrames."""

    print("\nComparaison des formats de colonnes entre les DataFrames:")
    print("------------------------------------------------------")

    # Vérifier si tous les DataFrames ont le même ensemble de colonnes
    noms_colonnes = None
    premiere_iteration = True

    for fichier, format_cols in formats_colonnes.items():
        colonnes_actuelles = set(format_cols.keys())
        if premiere_iteration:
            noms_colonnes = colonnes_actuelles
            premiere_iteration = False
        elif colonnes_actuelles != noms_colonnes:
            print(
                f"Erreur : Le fichier '{fichier}' a des colonnes "
                "différentes des autres DataFrames.")
            break
    else:
        # Si tous les DataFrames ont les mêmes colonnes, comparer les types
        premiere_iteration_types = True
        types_colonnes_reference = None

        for fichier, format_cols in formats_colonnes.items():
            if premiere_iteration_types:
                types_colonnes_reference = format_cols
                premiere_iteration_types = False
            elif format_cols != types_colonnes_reference:
                print(
                    f"Erreur : Le fichier '{fichier}' a des formats de "
                    "colonnes différents.")
                # Afficher les différences spécifiques pour ce fichier
                for colonne, type_ref in types_colonnes_reference.items():
                    if colonne in format_cols and format_cols[colonne] != type_ref:
                        print(
                            f"  - La colonne '{colonne}' a le format"
                            f"'{format_cols[colonne]}' dans '{fichier}' "
                            f"au lieu de '{type_ref}'."
                            )
                    elif colonne not in format_cols:
                        print(
                            f"  - La colonne '{colonne}' est manquante"
                            f"dans '{fichier}'."
                            )
                for colonne, type_act in format_cols.items():
                    if colonne not in types_colonnes_reference:
                        print(
                            f"  - La colonne '{colonne}' est présente dans"
                            f" '{fichier}' mais pas dans le premier DataFrame."
                            )
            else:
                print(
                    f"Le fichier '{fichier}' a le même format de colonnes "
                    "que les précédents."
                    )


In [ ]:
def modif_format(fichier, colonne, format, data_dict, formats_colonnes):
    """Modifie le format d'une colonne d'un DataFrame dans le dictionnaire."""

    data = data_dict[fichier]

    # On remplace les valeurs manquantes par 0 pour pouvoir changer le format
    data = data.fillna({colonne: 0})

    # Réassigner le résultat de astype à la colonne spécifiée
    data[colonne] = data[colonne].astype(format)

    # Mettre à jour le dictionnaire avec le DataFrame modifié
    data_dict[fichier] = data
    formats_colonnes[fichier] = data.dtypes.to_dict()

    return data_dict, formats_colonnes

In [ ]:
def concatenation(data_dict):
    """Effectue la concaténation de tous les DataFrames contenus
     dans le dictionnaire."""

    data_fin = pd.DataFrame()

    for data in data_dict.values():
        data_fin = pd.concat([data_fin, data], axis=0)

    return data_fin


In [ ]:
def ajout_annee(data, var_date):
    """
    Ajoute une colonne 'annee' au DataFrame correspondant à la composante
    année de "tourney_date".

    Args:
        data (pd.DataFrame):
            Le DataFrame contenant les données.
        var_date (str):
            Le nom de la colonne contenant les dates au format 'YYYYMMDD'.

    Returns:
        pd.DataFrame:
            Le DataFrame avec la colonne 'annee' ajoutée en première position.
    """
    # Conversion de int64 -> datetime
    data[var_date] = pd.to_datetime(
        data[var_date].astype(str),
        format="%Y%m%d",
        errors='coerce'
        )

    # Création de la colonne année
    data['annee'] = data[var_date].dt.year

    # Conversion en date simple (sans heure)
    data[var_date] = data[var_date].dt.date

    # Placer 'annee' en première position
    colonnes = ['annee'] + [col for col in data.columns if col != 'annee']
    data = data[colonnes]

    return data

In [ ]:
def afficher(data):
    """Affiche un descriptif des colonnes du DataFrame.

    Args:
        data (pd.DataFrame): Le DataFrame à analyser.
    """
    descriptif = pd.DataFrame({
        'Type': data.dtypes,
        'Nombre de valeurs manquantes': data.isnull().sum(),
        'Pourcentage de valeurs manquantes': (data.isnull().sum() / len(data)) * 100
    })
    display(descriptif)

## ATP

### matches simple

In [215]:
data_dict, formats_colonnes = recup_match("atp_matches","atp","_19","_20")

In [216]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'atp_matches_1969.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1969.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1970.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1971.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1971.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1972.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1973.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1973.csv' au lieu de 'float6

In [217]:
data_dict['atp_matches_1968.csv'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32.0,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32.0,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32.0,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32.0,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32.0,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_dict, formats_colonnes = modif_format(
    'atp_matches_1968.csv',
    'draw_size', 'int64',
    data_dict,
    formats_colonnes
    )

In [219]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1978.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1979.csv' a le même format de co

Concaténation

In [220]:
atp_matches_1968_2024 = concatenation(data_dict)

Selection des variables d'intérets

In [221]:
atp_matches_1968_2024 = atp_matches_1968_2024[var_interet]

In [222]:
atp_matches_1968_2024 = ajout_annee(atp_matches_1968_2024,'tourney_date')
afficher(atp_matches_1968_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.000000
tourney_date,object,0,0.000000
tourney_id,object,0,0.000000
tourney_name,object,0,0.000000
tourney_level,object,0,0.000000
surface,object,2990,1.533365
score,object,9,0.004615
round,object,0,0.000000
match_num,int64,0,0.000000
minutes,float64,99891,51.227205


Sauvegarde

In [223]:

atp_matches_1968_2024.to_csv("../Donnees/atp_matches_1968_2024.csv", index=False)

### matchs futures

In [224]:
data_dict, formats_colonnes = recup_match("atp_matches_futures","atp","_19","_20")

In [225]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_futures_1992.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1993.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1994.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1995.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1996.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1997.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1998.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1999.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2000.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2001.csv' a le même format de colon

In [226]:
data_dict['atp_matches_futures_2020.csv'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,101,144869,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,330.0,117.0,560.0,44.0
1,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,102,208262,NaN,WC,...,NaN,NaN,NaN,NaN,NaN,NaN,857.0,15.0,1670.0,1.0
2,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,103,105607,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,852.0,15.0,1892.0,1.0
3,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,104,106032,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,342.0,110.0,903.0,13.0
4,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,105,207678,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,371.0,97.0,1212.0,5.0


In [227]:
data_dict['atp_matches_futures_2019.csv'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,1,126135,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,618.0,3.0,338.0,67.0
1,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,2,200553,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,415.0,27.0,403.0,30.0
2,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,3,145001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,466.0,16.0,503.0,11.0
3,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,4,106187,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,359.0,57.0,413.0,28.0
4,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,5,134120,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354.0,58.0


On choisit de modifier la variables "tourney_level" en str

In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        "tourney_level",
        'str',
        data_dict,
        formats_colonnes
        )

In [229]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_futures_1992.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1993.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1994.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1995.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1996.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1997.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1998.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1999.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2000.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2001.csv' a le même format de colon

Concaténation

In [230]:
atp_matches_futures_1992_2024 = concatenation(data_dict)

In [231]:
atp_matches_futures_1992_2024 = atp_matches_futures_1992_2024[var_interet]

In [ ]:

atp_matches_futures_1992_2024 = ajout_annee(
    atp_matches_futures_1992_2024,
    'tourney_date'
    )
afficher(atp_matches_futures_1992_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.000000
tourney_date,object,0,0.000000
tourney_id,object,0,0.000000
tourney_name,object,0,0.000000
tourney_level,object,0,0.000000
surface,object,0,0.000000
score,object,33,0.006619
round,object,0,0.000000
match_num,int64,0,0.000000
minutes,float64,498555,100.000000


Sauvegarde

In [ ]:
atp_matches_futures_1992_2024.to_csv(
    "../Donnees/atp_matches_futures_1992_2024.csv",
    index=False)

### matchs qualificatifs

In [234]:
data_dict, formats_colonnes = recup_match("atp_matches_qual_chall","atp","_19","_20")

In [235]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_qual_chall_1979.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1980.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1981.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1982.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1983.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1984.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1985.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1986.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1987.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1988.

Concaténation

In [236]:
atp_matches_qual_1978_2024 = concatenation(data_dict)

In [237]:
atp_matches_qual_1978_2024 = atp_matches_qual_1978_2024[var_interet]

In [238]:
atp_matches_qual_1978_2024 = ajout_annee(atp_matches_qual_1978_2024,'tourney_date')
afficher(atp_matches_qual_1978_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.000000
tourney_date,object,0,0.000000
tourney_id,object,0,0.000000
tourney_name,object,0,0.000000
tourney_level,object,0,0.000000
surface,object,0,0.000000
score,object,77,0.034514
round,object,0,0.000000
match_num,int64,0,0.000000
minutes,float64,111056,49.779244


Sauvegarde

In [ ]:
atp_matches_qual_1978_2024.to_csv(
    "../Donnees/atp_matches_qual_1978_2024.csv",
    index=False)

### Rankings rangs

In [240]:
data_dict, formats_colonnes = recup_match("atp_rank","atp","","")

In [241]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_rankings_10s.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'atp_rankings_20s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'int64' dans 'atp_rankings_20s.csv' au lieu de 'float64'.
Le fichier 'atp_rankings_70s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_80s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_90s.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'atp_rankings_current.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'int64' dans 'atp_rankings_current.csv' au lieu de 'float64'.


In [242]:
data_dict['atp_rankings_10s.csv'].head()

,ranking_date,rank,player,points
0,20100104,1,103819,10550.0
1,20100104,2,104745,9205.0
2,20100104,3,104925,8310.0
3,20100104,4,104918,7030.0
4,20100104,5,105223,6785.0


In [243]:
data_dict['atp_rankings_20s.csv'].head()

,ranking_date,rank,player,points
0,20200106,1,104745,9985
1,20200106,2,104925,9055
2,20200106,3,103819,6590
3,20200106,4,106233,5825
4,20200106,5,106421,5705


In [ ]:
data_dict, formats_colonnes = modif_format(
    'atp_rankings_20s.csv',
    'points',
    'float64',
    data_dict,
    formats_colonnes
    )

data_dict, formats_colonnes = modif_format(
    'atp_rankings_current.csv',
    'points',
    'float64',
    data_dict,
    formats_colonnes
    )

In [245]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_rankings_10s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_20s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_70s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_80s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_90s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_current.csv' a le même format de colonnes que les précédents.


Concatenation

In [246]:
atp_rankings = concatenation(data_dict)

Selection des variables d'intérêts

In [247]:
atp_rankings = atp_rankings[["ranking_date", "rank", "player"]]

In [248]:
atp_rankings = ajout_annee(atp_rankings,'ranking_date')
afficher(atp_rankings)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.0
ranking_date,object,0,0.0
rank,int64,0,0.0
player,int64,0,0.0


Sauvegarde

In [249]:
atp_rankings.to_csv("../Donnees/atp_rankings.csv", index=False)

## WTA

### matches simple

In [250]:
data_dict, formats_colonnes = recup_match("wta_matches","wta","_19","_20")

In [251]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'wta_matches_1969.csv' a des formats de colonnes différents.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_1969.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'object' dans 'wta_matches_1970.csv' au lieu de 'float64'.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_1970.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_1971.csv' a des formats de colonnes différents.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_1971.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_

In [252]:
data_dict['wta_matches_1968.csv']["winner_entry"].unique()

array([nan])

In [253]:
data_dict['wta_matches_1969.csv']["winner_entry"].unique()

array([nan, 'LL', 'Q'], dtype=object)

In [ ]:
data_dict, formats_colonnes = modif_format(
    'wta_matches_1968.csv',
    "winner_entry",
    'str',
    data_dict,
    formats_colonnes
    )

In [255]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_1969.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'object' dans 'wta_matches_1970.csv' au lieu de 'float64'.
Le fichier 'wta_matches_1971.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
Erreur : Le fichier 'wta_matches_1973.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1973.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1973.csv' au

In [256]:
data_dict['wta_matches_1968.csv']["draw_size"].unique()

array([13., 11., 62., 96., 64.,  4., 32., 59., 16., 15., 31., 38., 23.,
        8., 55., 12., 29., 25., 27., 24., nan])

In [257]:
data_dict['wta_matches_1970.csv']["draw_size"].unique()

array(['43', '96', '64', '4', '16', '8', '12', '31', '32', '40', '27',
       '24', '48', '25', '28', '30', '6', '15', nan, 'exho'], dtype=object)

Ajout du type exho pour exhibition -> modification de toute les valeur en str

In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        "draw_size",
        'str',
        data_dict,
        formats_colonnes
        )


In [259]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1971.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
Erreur : Le fichier 'wta_matches_1973.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1973.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1973.csv' au lieu de 'object'.
Erreur : Le fichier 'wta_matches_1974.csv' a des formats de colonnes différen

In [260]:
data_dict['wta_matches_1971.csv']['winner_seed'].unique()

array([nan, '8', '5', '7', '1', '2', '3', '6', '4', '8F', '3F', '5F',
       '2F', '6F', '4F', '7F', '1F', '12', '14', '9', '10', '13', '15',
       '11'], dtype=object)

In [261]:
data_dict['wta_matches_1972.csv']['winner_seed'].unique()

array([ 1., nan,  8.,  5.,  4.,  3.,  6.,  7.,  2., 16.,  9., 12., 13.,
       14., 10., 15.])

In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        'winner_seed',
        'str',
        data_dict,
        formats_colonnes
        )

    data_dict, formats_colonnes = modif_format(
        fichier,
        'loser_seed',
        'str',
        data_dict,
        formats_colonnes
        )

In [263]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1978.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1979.csv' a le même format de co

Concaténation

In [264]:
wta_matches_1968_2024 = concatenation(data_dict)

In [265]:
wta_matches_1968_2024 = wta_matches_1968_2024[var_interet]

In [266]:
wta_matches_1968_2024 = ajout_annee(wta_matches_1968_2024,'tourney_date')
afficher(wta_matches_1968_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.000000
tourney_date,object,0,0.000000
tourney_id,object,0,0.000000
tourney_name,object,0,0.000000
tourney_level,object,0,0.000000
surface,object,4839,3.060876
score,object,1,0.000633
round,object,0,0.000000
match_num,int64,0,0.000000
minutes,float64,135965,86.003719


Enregistrement

In [267]:
wta_matches_1968_2024.to_csv("../Donnees/wta_matches_1968_2024.csv", index=False)

### matchs qualificatifs

In [268]:
data_dict, formats_colonnes = recup_match("wta_matches_qual","wta","","")

C:\Users\33663\AppData\Local\Temp\ipykernel_118912\2525029270.py:21: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)
C:\Users\33663\AppData\Local\Temp\ipykernel_118912\2525029270.py:21: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)
C:\Users\33663\AppData\Local\Temp\ipykernel_118912\2525029270.py:21: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)
C:\Users\33663\AppData\Local\Temp\ipykernel_118912\2525029270.py:21: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)


In [269]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'wta_matches_qual_itf_1969.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'float64' dans 'wta_matches_qual_itf_1969.csv' au lieu de 'int64'.
  - La colonne 'winner_seed' a le format 'object' dans 'wta_matches_qual_itf_1969.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_qual_itf_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'float64' dans 'wta_matches_qual_itf_1970.csv' au lieu de 'int64'.
  - La colonne 'winner_seed' a le format 'object' dans 'wta_matches_qual_itf_1970.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_qual_itf_1971.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'float64' dans 'wta_matches_qual_itf_1971.csv' au lieu de 'int64'.
  - La colonne 'loser_entry' a le format 'object' dans 'wta_matches_qual_itf_197

In [270]:
data_dict['wta_matches_qual_itf_1968.csv']['draw_size'].unique()

array([32, 16, 15, 31, 29, 96], dtype=int64)

In [271]:
data_dict['wta_matches_qual_itf_1968.csv']['winner_seed'].unique()

array([ 5., nan,  6.,  4.])

In [272]:
data_dict['wta_matches_qual_itf_1969.csv']['draw_size'].unique()

array([16., 32., 31., 30., nan, 64., 96.])

In [273]:
data_dict['wta_matches_qual_itf_1969.csv']['winner_seed'].unique()

array([nan, '4', '6', '5', '3', '7', '8', '2F'], dtype=object)

In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        'winner_seed',
        'str',
        data_dict,
        formats_colonnes
        )

    data_dict, formats_colonnes = modif_format(
        fichier,
        'draw_size',
        'int64',
        data_dict,
        formats_colonnes
        )

In [275]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_qual_itf_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1970.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_qual_itf_1971.csv' a des formats de colonnes différents.
  - La colonne 'loser_entry' a le format 'object' dans 'wta_matches_qual_itf_1971.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_qual_itf_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_qual_itf_1972.csv' au lieu de 'float64'.
  - La colonne 'loser_entry' a le format 'object' dans 'wta_matches_qual_itf_1972.csv' au lieu de 'float64'.
Le fichier 'wta_matches_qual_itf_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1974.csv' a le même format de colonnes que les précédents.
Erreur : L

In [276]:
data_dict['wta_matches_qual_itf_1968.csv']['loser_entry'].unique()

array([nan])

In [277]:
data_dict['wta_matches_qual_itf_1968.csv']['winner_entry'].unique()

array([nan])

In [278]:
data_dict['wta_matches_qual_itf_1972.csv']['loser_entry'].unique()

array([nan, 'Q'], dtype=object)

In [279]:
data_dict['wta_matches_qual_itf_1972.csv']['winner_entry'].unique()

array([nan, 'Q'], dtype=object)

In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        'winner_entry',
        'str',
        data_dict,
        formats_colonnes
        )

    data_dict, formats_colonnes = modif_format(
        fichier,
        'loser_entry',
        'str',
        data_dict,
        formats_colonnes
        )

In [281]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_qual_itf_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1978.csv' a le même forma

In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        'loser_seed',
        'str',
        data_dict,
        formats_colonnes
        )

In [283]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_qual_itf_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1978.csv' a le même forma

Concatenation

In [284]:
wta_matches_qual_1968_2024 = concatenation(data_dict)

In [285]:
wta_matches_qual_1968_2024 = wta_matches_qual_1968_2024[var_interet]

In [286]:
wta_matches_qual_1968_2024 = ajout_annee(wta_matches_qual_1968_2024,'tourney_date')
afficher(wta_matches_qual_1968_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.000000
tourney_date,object,0,0.000000
tourney_id,object,0,0.000000
tourney_name,object,0,0.000000
tourney_level,object,0,0.000000
surface,object,1650,0.277832
score,object,196,0.033003
round,object,0,0.000000
match_num,int64,0,0.000000
minutes,float64,576868,97.134630


Enregistrement

In [ ]:
wta_matches_qual_1968_2024.to_csv(
    "../Donnees/wta_matches_qual_1968_2024.csv",
    index=False)

### rankings

In [8]:
data_dict, formats_colonnes = recup_match("wta_rank","wta","","")

In [9]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'wta_rankings_10s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'float64' dans 'wta_rankings_10s.csv' au lieu de 'int64'.
Erreur : Le fichier 'wta_rankings_20s.csv' a des formats de colonnes différents.
  - La colonne 'tours' a le format 'int64' dans 'wta_rankings_20s.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_rankings_80s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'float64' dans 'wta_rankings_80s.csv' au lieu de 'int64'.
Erreur : Le fichier 'wta_rankings_90s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'float64' dans 'wta_rankings_90s.csv' au lieu de 'int64'.
Erreur : Le fichier 'wta_rankings_current.csv' a des formats de colonnes différents.
  - La colonne 'tours' a le format 'int64' dans 'wta_rankings_current.csv' au lieu de 'float64'.


In [ ]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(
        fichier,
        'points',
        'int64',
        data_dict,
        formats_colonnes
        )

    data_dict, formats_colonnes = modif_format(
        fichier,
        'tours',
        'int64',
        data_dict,
        formats_colonnes
        )


In [11]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_rankings_10s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_20s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_80s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_90s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_current.csv' a le même format de colonnes que les précédents.


Concaténation

In [12]:
wta_rankings = concatenation(data_dict)

Selection des variables d'intérêts

In [13]:
wta_rankings = wta_rankings[["ranking_date", "rank", "player"]]

In [14]:
wta_rankings = ajout_annee(wta_rankings,'ranking_date')
afficher(wta_rankings)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
annee,int32,0,0.0
ranking_date,object,0,0.0
rank,int64,0,0.0
player,int64,0,0.0


Enregistrement

In [295]:
wta_rankings.to_csv("../Donnees/wta_rankings.csv", index=False)